In [1]:
suppressPackageStartupMessages({
    source('../libs.R')
    source('../utils.R')    
#     source('../utils_mapping.R')
})


In [2]:
dname <- '../../data/cache/obj_fibroblasts/'
fields_load <- c('gsea', 'masc', 'masc_meta', 'effects_marginal', 'meta_data', 'effects_nested')
fib <- map(fields_load, function(name) {
    readRDS(file.path(dname, paste0(name, '.rds')))
})
names(fib) <- fields_load


# AHCA/TS Markers (*) 

In [3]:
## Adult Human Cell Atlas
effects_ahca <- readRDS('../../data/AdultHumanCellAtlas/cache/presto_cluster_nested.rds')
effects_ahca$padj <- p.adjust(effects_ahca$pvalue, 'BH')
effects_ahca$contrast <- gsub('mesenchymal', 'fibroblast', effects_ahca$contrast)
effects_ahca$Cluster <- gsub('mesenchymal', 'fibroblast', effects_ahca$Cluster)

effects_ts <- readRDS('../../data/tabula_sapiens/presto_cluster_nested.rds')
effects_ts$padj <- p.adjust(effects_ts$pvalue, 'BH')


In [4]:
genes_ts <- data.table(effects_ts)[
    ## marker is upregulated in all tissues for that cluster
    , sum(padj < .05 & beta > 0), by = .(Cluster, feature)
][V1 == 1 & Cluster == 'Stromal', feature]
genes_ahca <- data.table(effects_ahca)[
    ## marker is upregulated in all tissues for that cluster
    , sum(padj < .05 & beta > 0), by = .(Cluster, feature)
][V1 == 1 & Cluster == 'Stromal', feature]
genes <- union(genes_ts, genes_ahca)
length(genes)

[1] 936

In [5]:
.x_ts <- effects_ts %>% 
    subset(Cluster == 'Stromal') %>% 
    subset(beta > 0 & padj < .05) %>% 
    split(.$feature) %>% 
    map(function(.SD) {
        data.frame(
            tissues_ts = paste(.SD$Tissue, collapse = ';'),
            ntissues_ts = nrow(.SD)
        )
    }) %>% 
    bind_rows(.id = 'feature') %>% 
    dplyr::arrange(-ntissues_ts)


In [6]:
.x_ahca <- effects_ahca %>% 
    subset(Cluster == 'fibroblast') %>% 
    subset(beta > 0 & padj < .05) %>% 
    split(.$feature) %>% 
    map(function(.SD) {
        data.frame(
            tissues_ahca = paste(.SD$tissue, collapse = ';'),
            ntissues_ahca = nrow(.SD)
        )
    }) %>% 
    bind_rows(.id = 'feature') %>% 
    dplyr::arrange(-ntissues_ahca)


In [7]:
table_ts_ahca <- full_join(.x_ahca, .x_ts, by = 'feature') %>% 
    dplyr::arrange(-(ntissues_ts + ntissues_ahca)) 

head(table_ts_ahca)


,feature,tissues_ahca,ntissues_ahca,tissues_ts,ntissues_ts
,<chr>,<chr>,<int>,<chr>,<int>
1,ADAMTS12,Esophagus;Skin;Bladder;Rectum;Heart;Trachea;Muscle,7,Muscle;Large_Intestine;Lung;Trachea;Vasculature;Thymus;Bladder,7
2,ADCYAP1R1,Esophagus;Bladder;Rectum;Trachea;Heart;Skin;Muscle,7,Thymus;Large_Intestine;Bladder;Trachea;Lung;Muscle;Vasculature,7
3,AGTR1,Esophagus;Skin;Heart;Bladder;Trachea;Rectum;Muscle,7,Thymus;Large_Intestine;Lung;Trachea;Bladder;Muscle;Vasculature,7
4,ANGPTL5,Skin;Heart;Trachea;Esophagus;Rectum;Bladder;Muscle,7,Bladder;Large_Intestine;Lung;Muscle;Thymus;Trachea;Vasculature,7
5,CACNA2D1,Esophagus;Heart;Muscle;Trachea;Bladder;Rectum;Skin,7,Thymus;Trachea;Bladder;Lung;Vasculature;Muscle;Large_Intestine,7
6,CEP126,Esophagus;Skin;Heart;Muscle;Bladder;Rectum;Trachea,7,Thymus;Bladder;Vasculature;Trachea;Muscle;Large_Intestine;Lung,7


# Clinical tables

In [8]:
fname <- '../../manuscript/tables/clinical_tables.xlsx'
# rx_df <- readRDS('../../data/clinical/treatment_r2c23.rds')
clin_tables <- map(openxlsx::getSheetNames(fname), openxlsx::readWorkbook, xlsxFile = fname)
# %>% 
#     map(left_join, rx_df, by = 'LibraryID') 

map(clin_tables, head, 1)

[[1]]
  Sample.ID InternalID LibraryID Condition Sex Age Joint Seropositivity
1 Synovium1      BX233    BRI011        RA   M  43  knee   Seropositive
           Treatment
1 MTX15;HCQ200;LEF20

[[2]]
  SampleID InternalID LibraryID Diagnosis Age  Sex Serology Treatment
1    Lung1        207    BRI207   Control  NA <NA> negative      none
  Supplemental.O2 TLC DLCO Disease.Stage
1            <NA>  NA   NA          <NA>

[[3]]
        SampleID InternalID LibraryID Sex Diagnosis
1 SalivaryGland1          B      GX47   F       PSS
  Presence_of_antiRo_antibodies Focus_score                 Histology Treatment
1                          Neg         1.74 FLS. Mild focal fibrosis.      none

[[4]]
  SampleID   DonorID internalSampleID LibraryID  Status  YOB    Sex Nancy
1     Gut1 GutDonor1           GI2065    BRI021 Healthy 1998 female    NA
          Location Treatment
1 Ascending colon    unknown

## Add lung TLC/DLCO data

In [9]:
# clin_tables[[2]] <- clin_tables[[2]] %>% 
#     left_join(
#         fread('../../data/clinical/clinical_lung_tlc_dlco.txt')[, LibraryID := paste0('BRI', BRI)][, BRI := NULL][, .(LibraryID, SupplementalO2=Early, TLC, DLCO)],
#         by = 'LibraryID'
#     ) %>% 
#     dplyr::mutate(
#         SupplementalO2=case_when(
#             SupplementalO2 == 0 ~ 'No',
#             SupplementalO2 == 1 ~ 'Yes'
#         ),
#         TLC = as.integer(TLC),
#         DLCO = as.integer(DLCO)
#     )
    


## Anonymize for publishing

In [10]:
clin_tables_clean <- clin_tables %>% 
    map(dplyr::select, -LibraryID, -matches('^InternalID$|^internalSampleID$')) 

map(clin_tables_clean, head, 2)


[[1]]
  Sample.ID Condition Sex Age Joint Seropositivity          Treatment
1 Synovium1        RA   M  43  knee   Seropositive MTX15;HCQ200;LEF20
2 Synovium2        RA   M  77  knee   Seropositive      MTX20;lHCQ400

[[2]]
  SampleID Diagnosis Age  Sex Serology Treatment Supplemental.O2 TLC DLCO
1    Lung1   Control  NA <NA> negative      none            <NA>  NA   NA
2    Lung2   Control  NA <NA> negative      none            <NA>  NA   NA
  Disease.Stage
1          <NA>
2          <NA>

[[3]]
        SampleID Sex Diagnosis Presence_of_antiRo_antibodies Focus_score
1 SalivaryGland1   F       PSS                          Neg         1.74
2 SalivaryGland2   M       PSS                           Neg        2.78
                  Histology Treatment
1 FLS. Mild focal fibrosis.      none
2 FLS. Mild focal fibrosis.      none

[[4]]
  SampleID   DonorID  Status  YOB    Sex Nancy         Location Treatment
1     Gut1 GutDonor1 Healthy 1998 female    NA Ascending colon    unknown
2     Gut2 GutDonor2 Healthy 1946 female    NA    Sigmoid colon   unknown

In [11]:
clin_tables_clean[[2]]

,SampleID,Diagnosis,Age,Sex,Serology,Treatment,Supplemental.O2,TLC,DLCO,Disease.Stage
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,Lung1,Control,NA,NA,negative,none,NA,NA,NA,NA
2,Lung2,Control,NA,NA,negative,none,NA,NA,NA,NA
3,Lung3,Control,NA,NA,negative,none,NA,NA,NA,NA
4,Lung4,Control,NA,NA,negative,none,NA,NA,NA,NA
5,Lung5,RA-ILD,NA,NA,negative,none,Yes,NA,NA,Later
6,Lung6,IPF,61,F,negative,none,Yes,NA,NA,Later
7,Lung7,IPF,68,M,negative,Prednisone,Yes,42,23,Later
8,Lung8,Hypersensitivity Pneumonitis,53,F,negative,Prednisone,Yes,NA,NA,Later
9,Lung9,RA-ILD,67,M,positive,Mycophenolate Mofetil,Yes,NA,NA,Later


# Samples

Only post-QC 


In [12]:
# t1 <- readRDS('/data/srlab2/ik936/Roche/data/cache/all_obj_all/meta_data.rds') %>% 
#     dplyr::select(
#         CellID, Tissue, LibraryID, DonorID, 
#         Diagnosis, Technology, 
#         Lineage = joint_lineage, 
#         nUMI, nGene, percent_mito, 
#     ) 

# head(t1)


In [13]:
# t1 <- readRDS('/data/srlab2/ik936/Roche/data/cache/all_obj_all/meta_data.rds') %>% 
#     dplyr::mutate(
#         Diagnosis = case_when(
#             Tissue == 'Lung' ~ Diagnosis, 
#             TRUE ~ Case
#         )
#     ) %>% 
#     dplyr::select(
#         Tissue, LibraryID, DonorID, 
# #         Case, 
#         Diagnosis, 
#         Technology
#     ) %>% 
#     unique()

# head(t1)
# nrow(t1)


# inflammation scores

In [14]:
# t3 <- readRDS('/data/srlab2/ik936/Roche/data/cache/inflam_scores_icdf.rds') %>% 
#     dplyr::select(LibraryID, Tissue, InflamScore_raw = InflamScore, InflamScore = InflamScore_beta)

# head(t3)


# Within-tissue DGE (*) 

## Public markers

In [15]:
str <- 'DCN	CD74	ASPN	PRG4
IGF1	IFITM3	DKK3	FN1
FBLN1	LGALS3BP	MAMDC2	ITGB8
PDGFRL	HLA-DRA	ABHD2	CRTAC1
SFRP1	HLA-B	PRELP	HTRA1
C3	CXCL12	COL5A1	PCSK6
GAS6	HLA-DPA1	EMP1	HBEGF
FBLN5	HLA-DRB1	ANGPTL1	THBS4
RPL37	PLAU	CADM1	TIMP3
VCAN	HLA-A	FMOD	APLP2
RPL35A	IFI27	OGN	SMIM14
CD34	B2M	PDGFRB	NTN4
GSN	HLA-DPB1	CD109	CD55
SERPINF1	MDK	MYADM	CLIC5
PPAP2B	ZFP36L1	AHNAK	C10orf105
MFAP5	IFI30	VIM	ERRFI1
PODN	STAT1	COL8A2	CREB5
PTGFR	LOXL1	CAMK1D	BCAT1
SFRP2	BGN	DPT	FAM49A
CFH	C1S	COMP	ITGBL1
'


x <- strsplit(str, '\n') %>% map(strsplit, '\t') %>% map(data.frame)
x <- as.data.frame(t(x[[1]])) %>% as_tibble()
colnames(x) <- c('CD34+ sublining (SC-F1)', 'HLA-DRAhi sublining (SC-F2)', 'DKK3+ sublining (SC-F3)', 'Lining (SC-F4)')
gs_synovium <- x %>% as.list() %>% map(as.character)


In [16]:
gs_lip <- list()
gs_lip$CD34pos <- c('TNFSF13B', 'IL7', 'CD34', 'FAP', 'ICAM1', 'VCAM1', 'PDPN', 'PDGFRA', 'PDGFRB', 'CXCL12')
gs_lip$CD34neg <- c('LTB1', 'LTB2', 'CCL19', 'CCL21')


x <- openxlsx::readWorkbook('../../data/aviv_gut/1-s2.0-S0092867419307329-mmc2.xlsx', 'Stromal') %>% 
    subset(ident %in% c(
        'Inflammatory Fibroblasts',
        'Myofibroblasts',
        'RSPO3+',
        'WNT2B+ Fos-hi',
        'WNT2B+ Fos-lo 1',
        'WNT2B+ Fos-lo 2',
        'WNT5B+ 1',
        'WNT5B+ 2'
    )
) %>% 
    subset(log2fc > 0) %>% 
    identity()
# x <- data.table(x)[log2fc > 0, head(.SD[order(-log2fc)], 1), , by = gene]
gs_gut <- split(x$gene, x$ident)


In [17]:
gs_lung <- readRDS('../../data/cache/banovich_clusters_fibroblast_object.rds')$gs


## Compile genelists 

In [18]:
gs_df <- list(
    gs_synovium %>% map(data.frame) %>% bind_rows(.id = 'Cluster_public') %>% cbind(Tissue = 'Synovium') %>% dplyr::rename(Feature = `.x..i..`),
    gs_gut %>% map(data.frame) %>% bind_rows(.id = 'Cluster_public') %>% cbind(Tissue = 'Gut') %>% dplyr::rename(Feature = `.x..i..`),
    gs_lip %>% map(data.frame) %>% bind_rows(.id = 'Cluster_public') %>% cbind(Tissue = 'SalivaryGland') %>% dplyr::rename(Feature = `.x..i..`),
    gs_lung %>% map(data.frame) %>% bind_rows(.id = 'Cluster_public') %>% cbind(Tissue = 'Lung') %>% dplyr::rename(Feature = `.x..i..`)
) %>% 
bind_rows()


## Attach to presto results 

In [19]:
t6 <- list(
    SalivaryGland = readRDS('../../data/cache/salivarygland_fibroblasts_obj.rds')$presto$effects,
    Gut = readRDS('../..//data/cache/fibroblast_object_gut.rds')$presto$effects,
    Lung = readRDS('../..//data/cache/lung_fibroblasts_obj.rds')$presto$effects,
    Synovium = readRDS('../..//data/cache/synovium_fibroblasts_obj.rds')$presto$effects
) %>% 
    bind_rows(.id = 'Tissue') %>% 
    subset(!Cluster %in% 'NOTCH3+ perivascular') %>%
    dplyr::select(Cluster, Tissue, Feature = feature, LogFoldChange = beta, Sigma = sigma, Zscore = zscore, Pval = pvalue) %>% 
    subset(Pval < .1)


In [20]:
t6 <- data.table(left_join(t6, gs_df))[
    , .(Cluster_public = paste(Cluster_public, collapse = ';'))
    , by = .(Cluster, Tissue, Feature, LogFoldChange, Sigma, Zscore, Pval)
]

Joining, by = c("Tissue", "Feature")


In [21]:
head(t6)

Cluster,Tissue,Feature,LogFoldChange,Sigma,Zscore,Pval,Cluster_public
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
CCL19,SalivaryGland,HSPH1,1.476528,0.6288576,2.347953,0.009438457,NA
CD34,SalivaryGland,TNN,2.925866,1.2501593,2.340394,0.009631692,NA
CD34,SalivaryGland,ASPN,1.962962,0.8451165,2.322712,0.010097316,NA
CCL19,SalivaryGland,DNAJB1,1.569224,0.6870579,2.283976,0.011186463,NA
CD34,SalivaryGland,THBS4,2.046988,0.8965259,2.283245,0.011207971,NA
CCL19,SalivaryGland,RGS16,1.572704,0.6907502,2.276805,0.011398937,NA


# Gene selection stats

In [22]:
t7 <- readRDS('../../data/cache/iag_df.rds') %>% 
    dplyr::select(Tissue, Feature = feature, Slope = beta, Sigma = sigma, Zscore = zscore, Pval = pvalue) %>% 
    subset(Pval < .05)


In [23]:
head(t7)


Tissue,Feature,Slope,Sigma,Zscore,Pval
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
SalivaryGland,A1CF,1.5277086,0.6229209,2.452492,0.014187049
SalivaryGland,A2ML1,0.8811583,0.3533270,2.493889,0.012635195
SalivaryGland,AADACL2,1.0701482,0.4061467,2.634881,0.008416679
SalivaryGland,AATK,1.1757423,0.4724727,2.488487,0.012828779
SalivaryGland,ABCA1,0.5111586,0.2573504,1.986236,0.047007150
SalivaryGland,ABCA9,0.5403027,0.2714974,1.990085,0.046581626


# Joint fibroblast cluster marker stats

In [24]:
meta_data <- readRDS('../..//data/cache/obj_fibroblasts/meta_data.rds') 
effects_marginal <- readRDS('../..//data/cache/obj_fibroblasts/effects_marginal.rds') %>% 
    inner_join(unique(dplyr::select(meta_data, Cluster, Cluster_name))) %>% 
    dplyr::mutate(Tissue = 'Marginal') %>% 
    dplyr::select(Cluster = Cluster_name, Tissue, Feature = feature, LogFoldChange = beta, Sigma = sigma, Zscore = zscore, Pval = pvalue)

effects_nested <- readRDS('../..//data/cache/obj_fibroblasts/effects_nested.rds') %>% 
    inner_join(unique(dplyr::select(meta_data, Cluster, Cluster_name))) %>% 
    dplyr::select(Cluster = Cluster_name, Tissue, Feature = feature, LogFoldChange = beta, Sigma = sigma, Zscore = zscore, Pval = pvalue)

t8 <- rbind(effects_marginal, effects_nested) %>% 
    subset(Pval < .1)


Joining, by = "Cluster"
Joining, by = "Cluster"


In [25]:
head(t8)

Cluster,Tissue,Feature,LogFoldChange,Sigma,Zscore,Pval
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
CXCL10+CCL19+ C11,Marginal,CCL19,4.866968,0.6387375,7.619669,1.271637e-14
CD34+MFAP5+ C9,Marginal,PCOLCE2,2.606899,0.3801777,6.857055,3.514730e-12
CXCL10+CCL19+ C11,Marginal,CXCL10,4.535924,0.6722624,6.747252,7.533556e-12
CD34+MFAP5+ C9,Marginal,MFAP5,3.019067,0.4530927,6.663244,1.339244e-11
CD34+MFAP5+ C9,Marginal,C1QTNF3,2.525249,0.3883603,6.502337,3.954093e-11
CD34+MFAP5+ C9,Marginal,CYP4B1,1.912124,0.3295715,5.801849,3.279386e-09


# Joint fibroblast MASC results 

In [26]:
# t7 <- fib$masc %>% 
#     inner_join(unique(dplyr::select(meta_data, Cluster, Cluster_name))) %>% 
#     dplyr::select(Cluster = Cluster_name, Tissue, Ncells, LogFoldChange = beta, Sigma = sigma, Zscore = zscore, Pval = pvalue)

# head(t7)


# Pathways

In [27]:
t9 <- fib$gsea %>% 
#     dplyr::select(-leadingEdge) %>% 
    subset(database %in% c('H', 'C5')) %>% 
    subset(pval < .1)

head(t9, 1)


Cluster,database,pathway,pval,padj,ES,NES,nMoreExtreme,size,leadingEdge
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<list>
11,H,HALLMARK_ALLOGRAFT_REJECTION,0.0001968117,0.00294449,0.6968605,2.940547,0,157,"CCL19 , HLA-DRA , CD74 , IRF8 , CCL2 , CCL5 , CXCL9 , ICAM1 , CTSS , HLA-DQA1, HLA-DMB , CCL13 , HLA-DMA , HLA-DOB , STAT1 , TAP2 , IL15 , CD40 , ACHE , JAK2 , IL27RA , GBP2 , FYB1 , TRAF2 , THY1 , PSMB10 , RIPK2 , IL6 , SOCS1 , PTPRC , TAPBP , CCL7 , IRF7 , TAP1 , HLA-E , ITGB2 , CXCL13 , FAS , CCL11 , HLA-A , CSF1 , LIF , B2M , IL7 , CD7 , MMP9 , BCL3 , C2 , IFNAR2 , SRGN , CDKN2A , CD79A , TIMP1 , IL10 , NPM1 , IL4R , CD3G , IL12RB1 , LCP2 , NCK1 , RPS19 , HLA-DOA , IFNGR1 , RPS3A"


# Intra-cluster inflammation

In [28]:
t10 <- readRDS('../..//data/cache/R2C07_deseq2.rds') %>% 
    dplyr::mutate(Cluster = paste0('C', as.character(Cluster))) %>% 
    dplyr::select(Cluster, Tissue, feature, everything()) 

In [29]:
head(t10)

,Cluster,Tissue,feature,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<chr>,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,C5,SalivaryGland,A1BG,5.5847553,0.2824931126,1.2748163,0.221595148,0.8246291,NA
2,C5,SalivaryGland,A1CF,0.3722376,0.7515676972,4.8049740,0.156414521,0.8757063,NA
3,C5,SalivaryGland,A2M,58.6013326,-0.0008939626,0.6951188,-0.001286057,0.9989739,NA
4,C5,SalivaryGland,A2ML1,3.0177100,2.4507207785,1.8346846,1.335772257,0.1816237,NA
5,C5,SalivaryGland,A4GALT,101.0007561,-0.7599410737,0.6415354,-1.184566108,0.2361890,0.5089451
6,C5,SalivaryGland,AAAS,8.9402410,-0.7743134905,1.0299011,-0.751832843,0.4521516,NA


# Co-stim markers 

In [30]:
t11 <- readRDS('../..//data/cache/stim_revisions_ddres_v1.rds') %>% 
    dplyr::mutate(Tissue = c(fls = 'Synovium', lung = 'Lung')[Tissue])

In [31]:
head(t11)

,Tissue,Condition,feature,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Synovium,ECs,IGFBP2,5618.041,4.380813,0.1489800,29.40538,4.687768e-190,6.626628e-186
2,Synovium,ECs,S1PR3,1070.459,3.573479,0.1866052,19.14994,9.689077e-82,1.712060e-78
3,Synovium,ECs,NOTCH3,1084.259,3.266069,0.1733502,18.84087,3.491870e-79,5.484564e-76
4,Synovium,ECs,ERRFI1,3167.705,3.264059,0.1740926,18.74898,1.973353e-78,2.789532e-75
5,Synovium,ECs,DUSP6,1722.990,2.916591,0.1571357,18.56096,6.650267e-77,8.546198e-74
6,Synovium,ECs,TFPI,4122.846,3.101188,0.1717326,18.05824,6.795362e-73,8.004936e-70


# Co-stim pathways 

In [32]:
t12 <- fread('../..//manuscript/tables/EC_coculture_GO_pathways.csv', sep = ',')

In [33]:
head(t12)

Ontology,pathway,pval,padj,log2err,ES,NES,size,leadingEdge
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
BP,'de novo' posttranslational protein folding (GO:0051084),0.1124260,0.4985111,0.23112671,0.4085862,1.3117360,29,ERO1A|HSPA6|CCT2|ENTPD5|DNAJC7|HSPA8|HSPA9|CHCHD4|HSPE1|HSPA5|GAK|HSPA4|HSPA14|UGGT2|PTGES3
BP,2-oxoglutarate metabolic process (GO:0006103),0.9252336,0.9961201,0.03777050,-0.2510908,-0.6384680,17,ADHFE1|IDH1|AADAT|TAT
BP,3'-UTR-mediated mRNA destabilization (GO:0061158),0.8772504,0.9832574,0.04250232,-0.2988710,-0.6617787,10,CPEB3|ZC3H12D|RC3H1|ZFP36L2
BP,3'-UTR-mediated mRNA stabilization (GO:0070935),0.3027027,0.7155804,0.12878871,0.4381747,1.1298147,13,YBX3|ELAVL1|ZFP36|HNRNPC|DAZL|MAPKAPK2|TARDBP
BP,3'-phosphoadenosine 5'-phosphosulfate biosynthetic process (GO:0050428),0.2922374,0.7111497,0.11934844,0.5569713,1.1721255,6,PAPSS2|SLC35B2|SLC35B3
BP,3'-phosphoadenosine 5'-phosphosulfate metabolic process (GO:0050427),0.8157100,0.9651906,0.07511816,0.2506062,0.7732442,25,PODXL2|SULT4A1|PAPSS2|SLC35B2|TPST2|IMPAD1


# Dermal marker genes

In [34]:
## fibroblasts, mapped
t13 <- readRDS('../..//data/cache/obj_fib_eczema.rds')$query$effects %>% 
    dplyr::mutate(
        Tissue = 'Skin',
        Cluster = gsub('C', '', Cluster)
    ) %>% 
    inner_join(unique(dplyr::select(meta_data, Cluster, Cluster_name))) %>% 
    dplyr::select(Cluster = Cluster_name, Tissue, Feature = feature, LogFoldChange = beta, Sigma = sigma, Zscore = zscore, Pval = pvalue) %>% 
    subset(Pval < .1)


Joining, by = "Cluster"


In [35]:
head(t13)

Cluster,Tissue,Feature,LogFoldChange,Sigma,Zscore,Pval
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
SPARC+COL3A1+ C4,Skin,PLA2G2A,-2.938145,0.3671918,-8.001663,1.227500e-15
CD34+MFAP5+ C9,Skin,NKD2,-2.815291,0.3715960,-7.576216,3.557796e-14
CD34+MFAP5+ C9,Skin,COL4A1,-1.955736,0.2619006,-7.467471,8.175044e-14
CD34+MFAP5+ C9,Skin,FBN1,1.809090,0.2489661,7.266411,3.691647e-13
CD34+MFAP5+ C9,Skin,CD55,1.857448,0.2579580,7.200581,5.995667e-13
CD34+MFAP5+ C9,Skin,MFAP5,1.852379,0.2610586,7.095643,1.287515e-12


# Mouse marker genes 

In [36]:
# dname <- 
# # fields_load <- c('meta_data', 'effects_hs', 'U', 'masc')
# fields_load <- c('meta_data', 'effects_hs', 'presto_hs', 'U', 'masc')
# obj <- map(fields_load, function(name) {
#     readRDS(file.path(dname, paste0(name, '.rds')))
# })
# names(obj) <- fields_load


In [71]:
mm_presto <- readRDS('../..//data/cache/mouse_query_joint/presto_hs.rds') 


In [75]:
contrasts_mat <- make_contrast.presto(mm_presto, 'Cluster', 'Tissue')
contrasts_mat[is.na(contrasts_mat)] <- 0
t14 <- contrasts.presto(mm_presto, contrasts_mat, one_tailed = FALSE) %>% 
    tidyr::separate(contrast, c('Cluster', 'Tissue'), sep = '\\|', remove = FALSE) %>% 
#     arrange(pvalue)
    dplyr::mutate(Cluster = gsub('C', '', Cluster)) %>% 
    inner_join(unique(dplyr::select(meta_data, Cluster, Cluster_name))) %>% 
    dplyr::select(Cluster = Cluster_name, Tissue, Feature = feature, LogFoldChange = beta, Sigma = sigma, Zscore = zscore, Pval = pvalue) %>% 
    subset(Pval < .1)

Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 14 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14].”
Joining, by = "Cluster"


In [76]:
head(t14)


Cluster,Tissue,Feature,LogFoldChange,Sigma,Zscore,Pval
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
C1,Synovium_cia,A2M,3.0160142,0.9168663,3.289481,0.001003724
CXCL10+CCL19+ C11,Synovium_sta,AAMDC,-0.8550637,0.4018379,-2.127882,0.033346843
MYH11+ C13,Gut,AARD,-1.3430183,0.7476975,-1.796205,0.072461859
C6,Gut,AARD,1.6068837,0.7328393,2.192682,0.028330289
CD34+MFAP5+ C9,Gut,AARD,-1.3435749,0.7134690,-1.883158,0.059678958
C1,Lung,AARD,-1.0780220,0.5720651,-1.884439,0.059505574


# Mouse MASC

In [40]:
# t11 <- readRDS('/data/srlab2/ik936/Roche/data/cache/mouse_query_joint/masc.rds') %>% 
#     dplyr::mutate(Cluster = gsub('C', '', Cluster), Species = 'mm') %>% 
#     inner_join(unique(dplyr::select(meta_data, Cluster, Cluster_name))) %>% 
#     dplyr::select(Cluster = Cluster_name, Tissue, Species, Ncells, LogFoldChange = beta, Sigma = sigma, Zscore = zscore, Pval = pvalue)  


In [41]:
# head(t11)


# DSS phase modules enrichment

In [42]:
t15 <- readRDS('../..//data/cache/gut_time_enrichment.rds')


In [43]:
head(t15, 1)


Cluster,database,pathway,pval,padj,ES,NES,nMoreExtreme,size,leadingEdge
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<list>
13,1,1,0.0005181347,0.0009326425,-0.3818272,-2.084944,0,392,"Il6 , Cp , Cyp7b1 , Icam1 , Nfkbie , C6 , Hgf , Cxcl5 , Chil1 , Socs3 , Rasd1 , Irf1 , Limd2 , Sfrp1 , Sat1 , Cebpd , Irf8 , Il34 , Gpr68 , Glis1 , Csta1 , Medag , Ctsk , Cemip , Tnfrsf1b , Tnfaip2 , Cd40 , Ccl8 , Fscn1 , Adam12 , Cxcl10 , Rasl11a , Aldh1a3 , Rpl28 , Herc6 , Srpx2 , Trafd1 , Rhoh , Tmem140 , Lacc1 , F10 , Fcgr3 , Fcgr2b , Cyp1b1 , Elk3 , Tmem86a , Ly6e , Gadd45g , Tifa , Cyba , Rbp1 , Bst1 , Cxcl9 , Snx10 , Il16 , Slc40a1 , Ccr1 , Cxcl16 , Pear1 , Susd1 , Ms4a7 , Nfil3 , H2-Oa , Prr16 , Tmem176a , Cd22 , Lamc2 , Tmem229b , Hp , Tap1 , AB124611 , Ptx3 , S1pr2 , Il11 , Serpina3n, Serpina3i, Serpina3g, Serpina3f, Abcg3 , Il4ra , Gch1 , Thpo , Hk2 , Plekhs1 , Zmynd15 , Rps28 , Trim34a , Il12b , C1s1 , Mrc2 , Ido1 , Nrg1 , Sla , Etl4 , Coro1a , Mmp3 , Mmp10 , Igsf6 , Themis2 , Sgip1 , Slc15a3 , Pgd , Sulf1 , Lcn2 , Limk2 , Egfl7 , C1ra , Cd163l1 , Mid1 , Unc93b1 , Nxnl2 , Emb , Dmbt1 , Prf1 , Il12rb1 , Rac2 , Cfp , Ripk3 , Tnfrsf1a , Ifitm6 , Ifitm1 , Arid3a , Cd302 , Fmnl1 , B3gnt5 , Prkch , Olfml3 , Pros1 , Sirpb1b , Sirpa , Gm9733 , Gm5150 , Gpsm3 , Tmem71 , Cyp4f18 , Aoah , Itgax , Mttp , Ifnar2 , Slc11a1 , Myo1f , Draxin , Ncf1 , Timp1 , Stom"


# DSS-time sorted fibroblasts 

In [44]:
lres_dynamic <- readRDS('../..//data/cache/DSS_lres_dynamics.rds')
lres_peaks <- readRDS('../..//data/cache/DSS_lres_peaks.rds')
t16 <- lres_dynamic %>% 
    dplyr::select(-matches('^sm.ns')) %>% 
    left_join(dplyr::select(lres_peaks, feature, logFC, TimePoint)) %>% 
    tidyr::spread(TimePoint, logFC) %>% 
    dplyr::select(feature, AveExpr, F, P.Value, fdr, D2, D4, D7, D9, D11, D14)
    

Joining, by = "feature"


In [45]:
head(t16)

,feature,AveExpr,F,P.Value,fdr,D2,D4,D7,D9,D11,D14
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Mmp3,6.663360,25.88071,4.639500e-10,6.872027e-06,0.67485171,2.0044705,2.2321690,2.192098984,1.17392003,0.8614719
2,Aspn,6.882398,17.14975,6.836827e-08,3.423470e-04,-0.07962628,-0.1203999,-1.5844234,-1.218143558,0.06496953,-0.3026773
3,Lum,8.368560,17.11716,6.985597e-08,3.423470e-04,0.13147833,0.1952191,-0.5015135,-0.007835263,0.40966210,0.2958924
4,C3,7.016185,16.49894,1.056128e-07,3.423470e-04,-0.25414661,0.1767106,0.7544408,1.377849891,1.31230018,0.8241961
5,Actg2,2.729150,16.36615,1.155641e-07,3.423470e-04,-1.73553934,-1.2098486,1.9159592,2.562895704,3.33042266,2.1122172
6,Saa3,3.733894,16.02062,1.463908e-07,3.613902e-04,-0.21513325,2.9921403,5.4708013,5.093025282,2.06782075,2.2841579


# DSS GSEA 

In [46]:
t17 <- readRDS('../..//data/cache/DSS_enrich_res.rds')

In [47]:
head(t17, 1)

Cluster,pathway,pval,padj,log2err,ES,NES,size,leadingEdge
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<list>
C11,Inflammation,4.746245e-19,9.492491e-19,1.123915,0.831111,3.093872,58,"Gbp4 , Ccl2 , Cxcl9 , Saa3 , Plac8 , Gbp9 , Serpina3f , F830016B08Rik, Cxcl5 , Mt2 , Cxcl13 , Chl1 , Ifitm1 , Ifitm3 , Ccl7 , Cxcl1 , Mmp3 , Ccl19 , C3 , Gm12840 , Serpina3n , Angptl4 , Tmem176b , Cxcl2 , Serpina3m , Gm12185 , Ifitm2"


# Lung histology scores

In [48]:
t18 <- openxlsx::read.xlsx('../..//data/cache/marina_scores_final.xlsx') %>% 
    subset(!is.na(`Total.Lymphs.Per.50.40x.fields`) & !is.na(X2)) %>%
    left_join(clin_tables[[2]], by = 'LibraryID') %>% 
    dplyr::select(SampleID = SampleID.y, `Lymphocytes.per.10.40x.fields.(2mm2)`) %>% 
#     dplyr::select(SampleID = SampleID.y, `Total.Lymphs.Per.50.40x.fields`, `Lymphocytes.per.10.40x.fields.(2mm2)`) %>% 
    arrange(as.integer(gsub('Lung', '', SampleID)))


In [49]:
head(t18)

,SampleID,Lymphocytes.per.10.40x.fields.(2mm2)
,<chr>,<dbl>
1,Lung6,743.0
2,Lung8,538.0
3,Lung9,1145.0
4,Lung10,591.4
5,Lung11,1192.2
6,Lung12,798.0


# CellDive 

- Intensities
- fibroblast labels
- spots labels 


In [50]:
datadir <- '/n/data1/bwh/medicine/korsunsky/lab/'

lip <- list()
lip$spots <- readRDS(glue('{datadir}/lip1_spots.rds'))
lip$cells <- readRDS(glue('{datadir}/lip1_cells.rds'))
lip$fibroblasts <- readRDS(glue('{datadir}/lip1_fibroblasts.rds'))

gut <- list()
gut$spots <- readRDS(glue('{datadir}/gut_spots.rds'))
gut$cells <- readRDS(glue('{datadir}/gut_cells.rds'))
gut$fibroblasts <- readRDS(glue('{datadir}/gut_fibroblasts.rds'))

synovium <- list()
synovium$spots <- readRDS(glue('{datadir}/synovium_spots.rds'))
synovium$cells <- readRDS(glue('{datadir}/synovium_cells.rds'))
synovium$fibroblasts <- readRDS(glue('{datadir}/synovium_fibroblasts.rds'))


objs <- list(Gut = gut, Lip = lip, Synovium = synovium)

In [51]:
head(gut$cells$intensity)

,ASMA,CCL19,CD146,CD3,CD31,CD4,CD45,CD68,CD8,HLADR,PANCK,PDGFRA,PDPN,SPARC,CD90,VIM
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Cell1_GI6645,0,1190452,21939,644008,548743,201363,297550,922394,694126,634191,5284444,762560,103032,730023,440751,10597
Cell3_GI6645,0,1505727,1168,531237,412598,46768,219182,302885,394561,307304,4275421,792619,109070,563425,519665,2572
Cell4_GI6645,133637,1511036,214626,1302386,867329,5554252,2948766,5300883,1079104,3770658,429352,4771711,397016,1346791,839073,120826
Cell6_GI6645,7614,1657124,3314747,1635245,791974,3472400,2369609,3628397,804166,4598678,497518,1039844,410224,1576771,1821413,1100584
Cell7_GI6645,6181,466899,342504,362700,280757,1563638,356196,293002,186143,632992,184066,502334,100685,1066889,1870651,9725
Cell8_GI6645,19766,2522321,7904736,1821072,4802000,4490149,1624935,2464542,1001442,3463976,809246,3226949,400933,4136202,6418802,1327585


In [52]:
celldive_table <- objs %>% map('cells') %>% map('metadata') %>% 
    map(dplyr::select, CellID, LibraryID, x, y, x_img, y_img, area) %>% 
    bind_rows(.id = 'Tissue') %>% 
    left_join(
        objs %>% map('spots') %>% map('metadata') %>% 
            map(dplyr::select, CellID=SpotID, Niche=Niche_nice) %>% 
            bind_rows(), 
        by = c('CellID')
    ) %>% 
    left_join(
        objs %>% map('fibroblasts') %>% map('metadata') %>% 
            map(dplyr::select, CellID, Subtype) %>% 
            bind_rows(),
        by = 'CellID'
    ) %>% dplyr::mutate(Subtype = case_when(
        Subtype == 'Immuno' ~ 'CCL19+',
        Subtype == 'Vascular' ~ 'SPARC+',
        Subtype == 'Fibroblast' ~ 'Fibroblast',
        TRUE ~ 'Other'
    )) %>% 
    dplyr::select(CellID, Tissue, LibraryID, x_center=x_img, y_center=y_img, area, Niche, CellType = Subtype)

X <- objs %>% map('cells') %>% map('intensity') %>% bind_rows()
colnames(X) <- paste0('total_intensity_', colnames(X))
X <- tibble::rownames_to_column(X, 'CellID')


celldive_table <- left_join(celldive_table, X, by = 'CellID')


In [53]:
t(head(celldive_table))

,1,2,3,4,5,6
CellID,Cell1_GI6645,Cell3_GI6645,Cell4_GI6645,Cell6_GI6645,Cell7_GI6645,Cell8_GI6645
Tissue,Gut,Gut,Gut,Gut,Gut,Gut
LibraryID,GI6645,GI6645,GI6645,GI6645,GI6645,GI6645
x_center,54.70615,60.42922,64.77590,115.05436,123.56009,123.92663
y_center,4013.387,3941.485,4064.045,4049.690,3952.805,3991.013
area,667,883,1004,975,441,1213
Niche,Other,Other,Other,Other,Other,Other
CellType,Other,Other,Other,Other,Other,Other
total_intensity_ASMA,0,0,133637,7614,6181,19766
total_intensity_CCL19,1190452,1505727,1511036,1657124,466899,2522321


In [63]:
## Also add DAPI channels
df_dapi <- list(
    readRDS('../../data/CellDive/cache/lip1_cells_raw.rds'),
    readRDS('../../data/CellDive/cache/gut_cells_raw.rds'),
    readRDS('../../data/CellDive/cache/synovium_cells_raw.rds')
) %>% map(function(obj) {
    dplyr::select(obj$intensity, total_intensity_DAPI_INIT = DAPI_INIT, total_intensity_DAPI_FINAL = DAPI_FINAL) %>% 
        cbind(CellID = obj$metadata$CellID)
}) %>% 
    bind_rows()


In [66]:
celldive_table <- celldive_table %>% 
    left_join(df_dapi)

Joining, by = "CellID"


# Save

In [77]:
x <- list(table_ts_ahca, 
     clin_tables_clean[[1]], 
     clin_tables_clean[[2]], 
     clin_tables_clean[[3]], 
     clin_tables_clean[[4]], 
     t6, t7, t8, t9, t10, celldive_table, t11, t12, t13, t14, t15, t16, t17, t18
)

names(x) <- paste0('TableS', seq_len(length(x)))


In [78]:
pryr::object_size(x)

160,872,368 B

In [79]:
openxlsx::write.xlsx(x, '../..//manuscript/tables/SupplementaryTables.xlsx', overwrite = TRUE)
